In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import imread
import nltk
from imblearn.over_sampling import RandomOverSampler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, accuracy_score
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import codecs
from tqdm import tqdm
import shutil




from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import LSTM, GRU,SimpleRNN
from keras.layers import Dense, Activation, Dropout
from keras.layers import Embedding
from keras.layers import BatchNormalization
from keras.utils import to_categorical
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.callbacks import EarlyStopping
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold
from imblearn.over_sampling import SMOTE
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report, confusion_matrix, roc_curve, auc
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.stem.porter import PorterStemmer
from keras.callbacks import EarlyStopping

import re
from tensorflow.keras.preprocessing.text import one_hot
import warnings
warnings.filterwarnings("ignore")

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
train_cleaned_data = pd.read_csv("drive/MyDrive/Colab Notebooks/data/train_cleaned_rnn.csv")
test_cleaned_data = pd.read_csv("drive/MyDrive/Colab Notebooks/data/test_cleaned_rnn.csv")

X_train_data = train_cleaned_data['tweet_cleaned']
y_train_data = train_cleaned_data['label']

X_test_data = test_cleaned_data['tweet_cleaned']
y_test_data = test_cleaned_data['label']


### Textvorverarbeitung

In [13]:
def one_hot_enc(X_data, y_data):
  corpus = [str(text) for text in X_data]
  vocab_size = 5000

  #one hot encoding
  one_hot_dir = [one_hot(words,vocab_size) for words in corpus]

  embedded_layer = pad_sequences(one_hot_dir,padding = 'pre')
  return np.array(embedded_layer), np.array(y_data)

In [14]:
def tokenize(X_data, y_data, tokenizer=None, max_len=40):
    X_data = [str(x) for x in X_data]

    # # Wenn kein Tokenizer bereitgestellt wird, erstelle einen neuen
    if tokenizer is None:
        tokenizer = Tokenizer(num_words=None, oov_token="<OOV>")
        tokenizer.fit_on_texts(X_data)

    # # Tokenisierung
    X_data_sequences = tokenizer.texts_to_sequences(X_data)

    X_data_padded_sequences = pad_sequences(X_data_sequences, padding='post', maxlen=max_len)
    return X_data_padded_sequences, y_data, tokenizer


### Sampling

In [15]:
def random_over_sample(X_data, y_data):
  ros = RandomOverSampler(random_state=42)
  X = X_data.values.reshape(-1, 1)
  y = y_data.values

  X_resampled, y_resampled = ros.fit_resample(X, y)
  X_resampled = X_resampled[:, 0]
  return X_resampled, y_resampled


### Embeddings

In [16]:
def load_embeddings(file_path, embedding_dim, word_index):
    embedding_index = {}
    with open(file_path, encoding='utf-8') as f:
        for line in f:
            values = line.rstrip().split(' ')
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embedding_index[word] = coefs

    embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
    for word, i in word_index.items():
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

    return embedding_matrix


In [ ]:
import requests, zipfile, io
zip_file_url = "http://nlp.stanford.edu/data/glove.6B.zip"
r = requests.get(zip_file_url)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()

In [ ]:
import requests, zipfile, io
zip_file_url = "https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip"
r = requests.get(zip_file_url)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()

### Model

In [25]:
def train_gru(x_train, y_train, x_val, y_val, vocab_size, max_len, embedding_matrix=None, embedding_dim=200, epochs=20, batch_size=32):
    model = Sequential()
    if embedding_matrix is not None:
        model.add(Embedding(vocab_size, embedding_dim, weights=[embedding_matrix], input_length=max_len, trainable=False))
    else:
        model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
    model.add(SpatialDropout1D(0.3))
    model.add(GRU(300))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])


    history = model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(x_val, y_val))
    return model, history


In [35]:
def train_gru_optimized(x_train, y_train, x_val, y_val, vocab_size, max_len, embedding_matrix=None, embedding_dim=300, epochs=5, batch_size=32, class_weights=None):
    model = Sequential()
    if embedding_matrix is not None:
        model.add(Embedding(vocab_size, embedding_dim, weights=[embedding_matrix], input_length=max_len, trainable=False))
    else:
        model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
    model.add(SpatialDropout1D(0.4))
    model.add(GRU(256, return_sequences=False, dropout=0.3, recurrent_dropout=0.3))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy', 'Recall'])
    # model.compile(loss=focal_loss(alpha=0.5, gamma=2), optimizer=Adam(), metrics=['accuracy', 'Recall'])


    # Early Stopping Callback hinzufügen
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=3,
        restore_best_weights=True
    )

    history = model.fit(
        x_train,
        y_train,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(x_val, y_val),
        callbacks=[early_stopping]
    )
    return model, history

def focal_loss(gamma=2., alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        epsilon = tf.keras.backend.epsilon()
        y_pred = tf.clip_by_value(y_pred, epsilon, 1. - epsilon)
        y_true = tf.cast(y_true, tf.float32)
        alpha_t = y_true * alpha + (1 - y_true) * (1 - alpha)
        p_t = y_true * y_pred + (1 - y_true) * (1 - y_pred)
        loss = -alpha_t * tf.math.pow((1 - p_t), gamma) * tf.math.log(p_t)
        return tf.reduce_mean(loss)
    return focal_loss_fixed

### Evaluation

In [19]:
def evaluate_on_val(model, history, X_val, y_val, threshold=0.35):
    """
    Evaluates the model on validation data with detailed metrics and visualizes training performance.

    Parameters:
    - model: Trained model to evaluate.
    - history: Training history object.
    - X_val: Validation data (features).
    - y_val: Validation data (labels).
    - threshold: Decision threshold to classify an instance as positive.

    Returns:
    None
    """
    # Plot Training and Validation Metrics
    plt.figure(figsize=(12, 6))

    # Accuracy
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    # Loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.tight_layout()
    plt.show()

    # Evaluate Validation Data
    print("\nValidation Performance:")
    y_val_pred = (model.predict(X_val) > threshold).astype(int)

    accuracy = accuracy_score(y_val, y_val_pred)
    precision = precision_score(y_val, y_val_pred)
    recall = recall_score(y_val, y_val_pred)
    f1 = f1_score(y_val, y_val_pred)
    # roc_auc = roc_auc_score(y_val, model.predict(X_val))

    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    # print(f"ROC AUC: {roc_auc:.4f}")

    print("\nClassification Report:")
    print(classification_report(y_val, y_val_pred))

    # Confusion Matrix
    conf_matrix = confusion_matrix(y_val, y_val_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['No Hate Speech', 'Hate Speech'], yticklabels=['No Hate Speech', 'Hate Speech'])
    plt.title('Confusion Matrix - Validation Data')
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.show()


In [20]:
def evaluate_on_test(model, X_test, y_test, threshold=0.35):
    """
    Evaluates the model on an independent test dataset with detailed metrics.

    Parameters:
    - model: Trained model to evaluate.
    - X_test: Test data (features).
    - y_test: Test data (labels).
    - threshold: Decision threshold to classify an instance as positive.

    Returns:
    None
    """
    print("\nTest Performance:")
    y_test_pred = (model.predict(X_test) > threshold).astype(int)

    accuracy = accuracy_score(y_test, y_test_pred)
    precision = precision_score(y_test, y_test_pred)
    recall = recall_score(y_test, y_test_pred)
    f1 = f1_score(y_test, y_test_pred)

    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")

    print("\nClassification Report:")
    print(classification_report(y_test, y_test_pred))

    # Confusion Matrix
    conf_matrix = confusion_matrix(y_test, y_test_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Greens', xticklabels=['No Hate Speech', 'Hate Speech'], yticklabels=['No Hate Speech', 'Hate Speech'])
    plt.title('Confusion Matrix - Test Data')
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.show()


### Training/Evaluate

In [21]:
def one_hot_and_split_data(X_train_data, y_train_data, X_test_data, y_test_data):
  X_gru, y_gru = one_hot_enc(X_train_data, y_train_data)
  X_train_gru,X_val_gru,y_train_gru,y_val_gru=train_test_split(X_gru,y_gru,test_size=0.2,random_state=0)
  X_test_gru, y_test_gru = one_hot_enc(X_test_data, y_test_data)
  return X_train_gru, X_val_gru, y_train_gru, y_val_gru, X_test_gru, y_test_gru

In [22]:
def tokenize_and_split_data(X_train_data, y_train_data, X_test_data, y_test_data):
  X_gru, y_gru, tokenizer = tokenize(X_resampled, y_resampled)
  X_train_gru,X_val_gru,y_train_gru,y_val_gru=train_test_split(X_gru,y_gru,test_size=0.2,random_state=42)
  X_test_gru, y_test_gru, _= tokenize(X_test_data, y_test_data, tokenizer)
  return X_train_gru, X_val_gru, y_train_gru, y_val_gru, X_test_gru, y_test_gru, tokenizer

##### One-Hot

In [ ]:
X_train_onehot,X_val_onehot,y_train_onehot,y_val_onehot, X_test_onehot, y_test_onehot = one_hot_and_split_data(X_train_data, y_train_data, X_test_data, y_test_data)
model, history = train_gru(X_train_onehot, y_train_onehot, X_val_onehot, y_val_onehot, 5000, len(X_train_onehot[0]), None, embedding_dim=200, epochs=10, batch_size=32)
evaluate_on_val(model, history, X_val_onehot, y_val_onehot)
evaluate_on_test(model, X_test_onehot, y_test_onehot)

##### Tokenize

In [36]:
X_resampled, y_resampled = random_over_sample(X_train_data, y_train_data)
X_train_token,X_val_token,y_train_token,y_val_token, X_test_token, y_test_token, tokenizer = tokenize_and_split_data(X_resampled, y_resampled, X_test_data, y_test_data)
glove_embedding_matrix = load_embeddings('drive/MyDrive/Colab Notebooks/data/embeddings/glove.twitter.27B.200d.txt', 200, tokenizer.word_index)

In [ ]:
model, history = train_gru(X_train_token, y_train_token, X_val_token, y_val_token, len(tokenizer.word_index)+1, 40, glove_embedding_matrix, embedding_dim=200, epochs=20, batch_size=32)
evaluate_on_val(model, history, X_val_token, y_val_token)
evaluate_on_test(model, X_test_token, y_test_token)

In [ ]:
model_optimized, history_optimized = train_gru_optimized(X_train_token, y_train_token, X_val_token, y_val_token, len(tokenizer.word_index) + 1, 40, glove_embedding_matrix, embedding_dim=200, epochs=40, batch_size=32)
evaluate_on_val(model_optimized, history_optimized, X_val_token, y_val_token)
evaluate_on_test(model_optimized, X_test_token, y_test_token)

Epoch 1/40
867/948 ━━━━━━━━━━━━━━━━━━━━ 7s 98ms/step - Recall: 0.6325 - accuracy: 0.5078 - loss: 48.5487

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

# Preprocess additional features
def preprocess_additional_features(data, feature_name):
    # Extrahiere die Features (Listen von Strings oder Arrays)
    feature_list = [x if isinstance(x, list) else [] for x in data[feature_name]]
    mlb = MultiLabelBinarizer()
    features_encoded = mlb.fit_transform(feature_list)
    return features_encoded, mlb

# Preprocess Training-, Validation- und Test-Daten
hashtags_encoded, hashtags_mlb = preprocess_additional_features(X_train_data, 'hashtags')
emojis_encoded, emojis_mlb = preprocess_additional_features(X_train_data, 'emojis')

# Aufteilen der Features entsprechend den Token-Splits
hashtags_train, hashtags_val, emojis_train, emojis_val = train_test_split(
    hashtags_encoded, emojis_encoded, test_size=0.2, random_state=42
)

hashtags_test = hashtags_mlb.transform(X_test_data['hashtags'])
emojis_test = emojis_mlb.transform(X_test_data['emojis'])

def train_gru_with_additional_features(x_train_text, x_train_hashtags, x_train_emojis, y_train,
                                       x_val_text, x_val_hashtags, x_val_emojis, y_val,
                                       vocab_size, max_len, embedding_matrix=None, embedding_dim=200,
                                       epochs=20, batch_size=32):
    # Text Input
    text_input = Input(shape=(max_len,), name="Text_Input")
    if embedding_matrix is not None:
        embedding = Embedding(vocab_size, embedding_dim, weights=[embedding_matrix], input_length=max_len, trainable=False)(text_input)
    else:
        embedding = Embedding(vocab_size, embedding_dim, input_length=max_len)(text_input)
    text_dropout = SpatialDropout1D(0.4)(embedding)
    text_rnn = Bidirectional(GRU(256, return_sequences=False, dropout=0.3, recurrent_dropout=0.3))(text_dropout)

    # Hashtags Input
    hashtags_input = Input(shape=(x_train_hashtags.shape[1],), name="Hashtags_Input")
    hashtags_dense = Dense(128, activation="relu")(hashtags_input)

    # Emojis Input
    emojis_input = Input(shape=(x_train_emojis.shape[1],), name="Emojis_Input")
    emojis_dense = Dense(64, activation="relu")(emojis_input)

    # Combine all inputs
    combined = Concatenate()([text_rnn, hashtags_dense, emojis_dense])
    dense = Dense(128, activation="relu")(combined)
    output = Dense(1, activation="sigmoid")(dense)

    # Define the model
    model = Model(inputs=[text_input, hashtags_input, emojis_input], outputs=output)
    model.compile(
        loss='binary_crossentropy',
        optimizer=Adam(learning_rate=0.001),
        metrics=['accuracy', 'Recall']
    )

    # Early Stopping Callback
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=3,
        restore_best_weights=True
    )

    # Train the model
    history = model.fit(
        [x_train_text, x_train_hashtags, x_train_emojis],
        y_train,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=([x_val_text, x_val_hashtags, x_val_emojis], y_val),
        callbacks=[early_stopping]
    )

    return model, history


In [ ]:
# Modell trainieren mit zusätzlichen Features
model_optimized, history_optimized = train_gru_with_additional_features(
    x_train_text=X_train_token,
    x_train_hashtags=hashtags_train,
    x_train_emojis=emojis_train,
    y_train=y_train_token,
    x_val_text=X_val_token,
    x_val_hashtags=hashtags_val,
    x_val_emojis=emojis_val,
    y_val=y_val_token,
    vocab_size=len(tokenizer.word_index) + 1,
    max_len=40,
    embedding_matrix=glove_embedding_matrix,
    embedding_dim=200,
    epochs=40,
    batch_size=32
)
